# Read and Store Labels

In [2]:
import tensorflow
import os
import tarfile
import cv2
import h5py
import numpy as np
import imageio
from PIL import Image
import matplotlib.pyplot as plt
from PIL import ImageOps
import pandas as pd
from skimage.util import img_as_ubyte
import csv
from numpy import savez_compressed
from numpy import load

In [3]:
# get the label of one picture
def get_one_label(data_path, index, debug=False): 
    #get_img_bbox(data_path ="/data/test/digitStruct.mat",index)
    attrs = {}
    data = h5py.File(data_path, 'r')
    item = data['digitStruct']['bbox'][index].item()
    key_list = ['label']
    for key in key_list:
        attr = data[item][key]
        values = []
        for i in range(len(attr)):
            # if multi-digits in the image, get infomation digit by digit
            if len(attr) > 1:
                values.append(data[attr.value[i].item()].value[0][0])
            # if only one digit in the image
            else:
                values = [attr.value[0][0]]
        attrs[key] = values[:5]
    return attrs['label']

In [4]:
# get all labels
def labels(path_img, path_mat, debug=False):
    labels = list()
    mat_file = h5py.File(os.getcwd()+path_mat,'r')
    bbox_df = pd.DataFrame([],columns=['img_name','label','left','top','width','height'])
    for i in range(mat_file['/digitStruct/bbox'].shape[0]):
        label = get_one_label(os.getcwd()+path_mat,i,debug=debug)
        #print(label)
        label = [0.0 if x==10 else x for x in label]
        lenth = len(label)
        if lenth < 5:
            add_list = [10.0]*(5-lenth)
            label.extend(add_list)
        label.insert(0, lenth)
        labels.append(np.array(label))
    labels = np.asarray(labels)
    return labels

In [1]:
# get and store test labels
test_label = labels('/data/test','/data/test/digitStruct.mat',debug=False)
savez_compressed(os.path.join(os.getcwd()+'/data/','test_label2.npz'),test_label)

In [63]:
# get and store train labels
train_label = labels('/data/train','/data/train/digitStruct.mat',debug=False)
savez_compressed(os.path.join(os.getcwd()+'/data/','train_label2.npz'),train_label)

In [6]:
# get and store extra labels
extra_label = labels('/data/extra','/data/extra/digitStruct.mat',debug=False)
savez_compressed(os.path.join(os.getcwd()+'/data/','extra_label2.npz'),extra_label)